In [242]:
import numpy as np
import torch
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split

In [243]:
data = pd.read_csv("train.csv")
y = data['SalePrice']
X = data.drop('SalePrice', axis=1)
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [244]:
X.drop('Id', axis=1)
numeric_cols = list(X.select_dtypes(include=np.number).columns)
X_num = X[numeric_cols]
X_num.columns = X[numeric_cols].columns
X_num.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,836,192,84,0,0,0,0,0,12,2008


## First model will use only numeric values and missing values will be imputed with SimpleImputer

In [245]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()

def get_train_test_data(X, y, test_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
    imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))
    
    imputed_X_train.columns = X_train.columns
    imputed_X_test.columns = X_test.columns
    
    X_train, X_test = imputed_X_train, imputed_X_test
    
    X_train = torch.tensor(X_train.values, dtype=torch.float32)
    X_test = torch.tensor(X_test.values, dtype=torch.float32)
    y_train = torch.tensor(y_train.values, dtype=torch.float32).squeeze()
    y_test = torch.tensor(y_test.values, dtype=torch.float32).squeeze()
    return X_train, X_test, y_train, y_test

In [246]:
X_train, X_test, y_train, y_test = get_train_test_data(X_num, y, test_size=0.2)
X_num = pd.DataFrame(my_imputer.transform(X_num))

In [247]:
class ModelV1(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=1),
            # nn.Linear(in_features=10, out_features=10),
            # nn.Linear(in_features=10, out_features=1)
        )
        
    def forward(self, x):
        return self.layers(x)

In [248]:
model1 = ModelV1(in_features=37)

loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model1.parameters(), lr=0.0001)

In [249]:
model1.eval()
y_pred = model1(X_train).squeeze()
y_pred

tensor([-533.5696, -687.5678, -429.7567,  ..., -836.2488, -845.8161,
        -626.0494], grad_fn=<SqueezeBackward0>)

In [250]:
def train_test(model:torch.nn.Module, loss_fn:torch.nn.Module, optimizer:torch.optim.Optimizer, epochs:int, X_train, X_test, y_train, y_test):
    
    for epoch in range(epochs):
        model.train()
        y_preds = model(X_train).squeeze()
        loss = loss_fn(y_preds, y_train)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        model.eval()
        with (torch.inference_mode()):
            test_pred = model(X_test).squeeze()
            test_loss = loss_fn(test_pred, y_test)
        if epoch % 50 == 0:
            print("----------------------------")
            print(f"Epoch {epoch}:")
            print(f"Train loss: {loss} | Test loss: {test_loss}")

In [251]:
train_test(model=model1, loss_fn=loss_fn, optimizer=optimizer, epochs=2000, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 182436.15625 | Test loss: 166294.09375
----------------------------
Epoch 50:
Train loss: 59787.87109375 | Test loss: 58783.5078125
----------------------------
Epoch 100:
Train loss: 55803.24609375 | Test loss: 54570.86328125
----------------------------
Epoch 150:
Train loss: 52470.28125 | Test loss: 51039.41796875
----------------------------
Epoch 200:
Train loss: 49978.33984375 | Test loss: 48293.703125
----------------------------
Epoch 250:
Train loss: 48100.69140625 | Test loss: 46118.17578125
----------------------------
Epoch 300:
Train loss: 46602.33203125 | Test loss: 44380.48046875
----------------------------
Epoch 350:
Train loss: 45474.86328125 | Test loss: 43123.46484375
----------------------------
Epoch 400:
Train loss: 44562.43359375 | Test loss: 42121.75
----------------------------
Epoch 450:
Train loss: 43754.6015625 | Test loss: 41192.265625
----------------------------
Epoch 500:
Train loss: 43026.66015625 | Tes

## This results are not great, let's figure out which 5 data columns have the most corelation with SalePrice and use only them

In [252]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [253]:
discrete_features = X_num.dtypes == int

mi_scores = make_mi_scores(X_num, y, discrete_features=discrete_features)
mi_scores[:7]

4     0.568731
16    0.483868
12    0.369258
6     0.363817
26    0.363205
27    0.359972
13    0.310404
Name: MI Scores, dtype: float64

## Let's now use only this 7 features in a Model and see if there is any difference in performance

In [254]:
model2 = ModelV1(in_features=7)
lista = list(mi_scores[:7].index)
for i in lista:
    i -= 1
X_num_2 = X_num[lista]
X_train, X_test, y_train, y_test = get_train_test_data(X_num_2, y, test_size=0.2)
optimizer_2 = torch.optim.SGD(params=model2.parameters(), lr=0.001)

In [255]:
train_test(model=model2, loss_fn=loss_fn, optimizer=optimizer_2, epochs=2000, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 179759.234375 | Test loss: 168484.21875
----------------------------
Epoch 50:
Train loss: 38941.55078125 | Test loss: 36424.44921875
----------------------------
Epoch 100:
Train loss: 37194.15234375 | Test loss: 34685.140625
----------------------------
Epoch 150:
Train loss: 35768.69921875 | Test loss: 33292.59765625
----------------------------
Epoch 200:
Train loss: 34592.20703125 | Test loss: 32282.595703125
----------------------------
Epoch 250:
Train loss: 33646.3203125 | Test loss: 31482.119140625
----------------------------
Epoch 300:
Train loss: 32891.6328125 | Test loss: 30847.267578125
----------------------------
Epoch 350:
Train loss: 32348.75390625 | Test loss: 30399.876953125
----------------------------
Epoch 400:
Train loss: 31954.017578125 | Test loss: 30042.16796875
----------------------------
Epoch 450:
Train loss: 31665.53125 | Test loss: 29770.03515625
----------------------------
Epoch 500:
Train loss: 31422.

## Now let's try to normalize data and see whether it improves our result or not

In [256]:
X_num_3 = X_num_2
y_normalized = pd.DataFrame(torch.nn.functional.normalize(torch.tensor(y, dtype=torch.float32).unsqueeze(dim=1)))
X_train, X_test, y_train, y_test = get_train_test_data(X_num_3, y_normalized, test_size=0.2)
X_train, X_test = torch.nn.functional.normalize(X_train), torch.nn.functional.normalize(X_test)

In [257]:
model3 = ModelV1(in_features=7)
optimizer_3 = torch.optim.SGD(params=model3.parameters(), lr=0.001)

In [258]:
train_test(model=model3, loss_fn=loss_fn, optimizer=optimizer_3, epochs=2001, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 1.082661509513855 | Test loss: 1.081758737564087
----------------------------
Epoch 50:
Train loss: 0.9845773577690125 | Test loss: 0.983674168586731
----------------------------
Epoch 100:
Train loss: 0.8864930272102356 | Test loss: 0.8855893611907959
----------------------------
Epoch 150:
Train loss: 0.7884080410003662 | Test loss: 0.7875040769577026
----------------------------
Epoch 200:
Train loss: 0.6903231143951416 | Test loss: 0.6894187927246094
----------------------------
Epoch 250:
Train loss: 0.5922382473945618 | Test loss: 0.5913335084915161
----------------------------
Epoch 300:
Train loss: 0.4941532611846924 | Test loss: 0.49324819445610046
----------------------------
Epoch 350:
Train loss: 0.3960684537887573 | Test loss: 0.395162969827652
----------------------------
Epoch 400:
Train loss: 0.2979835569858551 | Test loss: 0.2970777451992035
----------------------------
Epoch 450:
Train loss: 0.1998986303806305 | Test l

## Let's try now with more complicated model

In [259]:
X_train, X_test, y_train, y_test = get_train_test_data(X_num, y, test_size=0.2)

In [260]:
class ModelV2(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=10),
            nn.Linear(in_features=10, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )
        
    def forward(self, x):
        return self.layer(x)

In [261]:
model4 = ModelV2(37)
loss_fn = nn.L1Loss()
optimizer_4 = torch.optim.Adam(model4.parameters(), lr=0.01, weight_decay=1e-5)

In [262]:
train_test(model4, loss_fn=loss_fn, optimizer=optimizer_4, epochs=2000, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 180185.234375 | Test loss: 181588.0625
----------------------------
Epoch 50:
Train loss: 48170.96875 | Test loss: 54422.11328125
----------------------------
Epoch 100:
Train loss: 40470.79296875 | Test loss: 46123.33984375
----------------------------
Epoch 150:
Train loss: 32701.75390625 | Test loss: 37445.71875
----------------------------
Epoch 200:
Train loss: 27255.78125 | Test loss: 32690.876953125
----------------------------
Epoch 250:
Train loss: 26295.34765625 | Test loss: 32284.373046875
----------------------------
Epoch 300:
Train loss: 26095.6328125 | Test loss: 32142.9765625
----------------------------
Epoch 350:
Train loss: 25983.078125 | Test loss: 31996.14453125
----------------------------
Epoch 400:
Train loss: 25901.673828125 | Test loss: 31825.068359375
----------------------------
Epoch 450:
Train loss: 25845.037109375 | Test loss: 31592.794921875
----------------------------
Epoch 500:
Train loss: 25801.144531

## Let's now try using categorical variables

In [263]:
from sklearn.preprocessing import OrdinalEncoder

label_X = X.copy()

s = (X.dtypes == 'object')
object_cols = list(s[s].index)

ordinal_encoder = OrdinalEncoder()
label_X[object_cols] = ordinal_encoder.fit_transform(X[object_cols])
label_X = label_X.drop('Id', axis=1)
label_X = pd.DataFrame(my_imputer.fit_transform(label_X))
label_X.head()

,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
0,60.0,3.0,65.0,8450.0,1.0,0.450549,3.0,3.0,0.0,4.0,...,0.0,0.0,1.142857,1.427046,1.907407,0.0,2.0,2008.0,8.0,4.0
1,20.0,3.0,80.0,9600.0,1.0,0.450549,3.0,3.0,0.0,2.0,...,0.0,0.0,1.142857,1.427046,1.907407,0.0,5.0,2007.0,8.0,4.0
2,60.0,3.0,68.0,11250.0,1.0,0.450549,0.0,3.0,0.0,4.0,...,0.0,0.0,1.142857,1.427046,1.907407,0.0,9.0,2008.0,8.0,4.0
3,70.0,3.0,60.0,9550.0,1.0,0.450549,0.0,3.0,0.0,0.0,...,0.0,0.0,1.142857,1.427046,1.907407,0.0,2.0,2006.0,8.0,0.0
4,60.0,3.0,84.0,14260.0,1.0,0.450549,0.0,3.0,0.0,2.0,...,0.0,0.0,1.142857,1.427046,1.907407,0.0,12.0,2008.0,8.0,4.0


In [264]:
mi_scores_all = make_mi_scores(label_X, y, discrete_features=label_X.columns)
mi_scores_all[:10]

16    0.573534
11    0.532615
61    0.494348
45    0.432078
18    0.410554
37    0.399014
3     0.393403
60    0.368253
58    0.345855
26    0.332348
Name: MI Scores, dtype: float64

In [265]:
useful_data = list(mi_scores_all[mi_scores_all.gt(0.2)].index)
label_X = label_X[useful_data]

In [266]:
X_train, X_test, y_train, y_test = get_train_test_data(label_X, y, test_size=0.2)

In [293]:
model5 = ModelV2(len(useful_data))
optimizer_5 = torch.optim.Adam(params=model5.parameters(), lr=0.02, weight_decay=1e-4)

In [294]:
train_test(model5, loss_fn=loss_fn, optimizer=optimizer_5, epochs=2001, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 182317.1875 | Test loss: 176460.765625
----------------------------
Epoch 50:
Train loss: 50928.8828125 | Test loss: 51278.4453125
----------------------------
Epoch 100:
Train loss: 39822.203125 | Test loss: 35593.58203125
----------------------------
Epoch 150:
Train loss: 30334.123046875 | Test loss: 27949.6484375
----------------------------
Epoch 200:
Train loss: 29332.9140625 | Test loss: 26845.59375
----------------------------
Epoch 250:
Train loss: 29056.873046875 | Test loss: 26638.46484375
----------------------------
Epoch 300:
Train loss: 28936.041015625 | Test loss: 26552.19921875
----------------------------
Epoch 350:
Train loss: 28853.2734375 | Test loss: 26344.345703125
----------------------------
Epoch 400:
Train loss: 28758.630859375 | Test loss: 26264.69921875
----------------------------
Epoch 450:
Train loss: 28646.9921875 | Test loss: 26279.646484375
----------------------------
Epoch 500:
Train loss: 28550.3945

## Now let's make some predictions and see what score this model gets in competition

In [295]:
X_eval = pd.read_csv("test.csv")
X_eval.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal
1458,2919,60,RL,74.0,9627,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,11,2006,WD,Normal


In [296]:
label_X_test = X_eval.copy()
label_X_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [297]:
label_X_test = label_X_test.drop("Id", axis=1)

label_X_test[object_cols] = ordinal_encoder.fit_transform(label_X_test[object_cols])
label_X_test = pd.DataFrame(my_imputer.fit_transform(label_X_test))

label_X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
0,20.0,2.0,80.0,11622.0,1.0,0.345794,3.0,3.0,0.0,4.0,...,120.0,0.0,0.333333,2.000000,1.843137,0.0,6.0,2010.0,8.0,4.0
1,20.0,3.0,81.0,14267.0,1.0,0.345794,0.0,3.0,0.0,0.0,...,0.0,0.0,0.333333,1.396552,0.000000,12500.0,6.0,2010.0,8.0,4.0
2,60.0,3.0,74.0,13830.0,1.0,0.345794,0.0,3.0,0.0,4.0,...,0.0,0.0,0.333333,2.000000,1.843137,0.0,3.0,2010.0,8.0,4.0
3,60.0,3.0,78.0,9978.0,1.0,0.345794,0.0,3.0,0.0,4.0,...,0.0,0.0,0.333333,1.396552,1.843137,0.0,6.0,2010.0,8.0,4.0
4,120.0,3.0,43.0,5005.0,1.0,0.345794,0.0,1.0,0.0,4.0,...,144.0,0.0,0.333333,1.396552,1.843137,0.0,1.0,2010.0,8.0,4.0


In [298]:
label_X_test = torch.tensor(label_X_test[useful_data].values, dtype=torch.float32)

In [299]:
label_X_test.shape

torch.Size([1459, 22])

In [300]:
model5.eval()
y_res = model5(label_X_test)
predictions = y_res.detach().numpy()
predictions

array([[132689.2 ],
       [164598.05],
       [184803.9 ],
       ...,
       [146602.31],
       [112829.08],
       [221621.64]], dtype=float32)

In [301]:
import csv
def write_to_csv(predictions):
    with open("predicts.csv", mode="w") as f:
        fieldnames = [
            "Id",
            "SalePrice"
        ]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(0, len(predictions)):
            dct = {"Id":i+1461, "SalePrice":predictions[i][0]}
            writer.writerow(dct)

In [302]:
write_to_csv(predictions=predictions)

## This model has achieved score of 0.18515